## DEPENDENCE INSTALL

- Dependencies install for evaluation of metrics
- Instalação das depedendencias necessárias para executar todos as avaliações

In [ ]:
%pip install numpy cython
%pip install wheel
%pip install ir_datasets
%pip install scikit-learn
%pip install nltk
%pip install requests
%pip install pandas

In [12]:
import re
import ir_datasets
import numpy as np
import requests
import json
import nltk
from nltk.tokenize import word_tokenize

In [13]:
dataset = ir_datasets.load('cranfield')

In [3]:
dataset.docs_count()

1400

## INDEXAR DOCUMENTOS

In [ ]:
count = 0
nltk.download("punkt")
for doc in dataset.docs_iter():

  sents = list(word_tokenize(doc.text))

  mystr = ' '.join(map(str, sents))

  myobj = {
            'id': doc.doc_id,
            'title': doc.title.strip(),
            'body': mystr.strip()
          }
  response = requests.post('http://127.0.0.1:9200/cranfield/_doc/' + doc.doc_id, json=myobj)
  if response.status_code == 201:
    count += 1
  else:
    print("DOC ID ", doc.doc_id, "Status ", response.status_code)
    print(response.text)

print("Indexed ", count)

## EFETUAR BUSCAS

In [14]:
count = 0
documents = []
for query in dataset.queries_iter():

  # sent = list(word_tokenize(query.text))
  # mystr = ' '.join(map(str, sent))
  qry = "http://127.0.0.1:9200/cranfield/_doc/_search"
  x = requests.get(qry, json={
      "query": {
          "match": {
              "body": query.text
          }
      }
  })

  if x.status_code == 200:
    obj = json.loads(x.text)
    documents.append({
        "query_id":query.query_id,
        "result":obj
    })
    count += 1


print("Total..: ",count)
  # print("Query ID:", query.query_id, "Status code:", x.status_code)

Total..:  225


# RELEVANCIA

In [15]:
import pandas as pd
import functions.evaluetion_functions as ev
import functions.documents_process as dp


metrics = ev.generate_metrics_dcg_k(dataset, documents, "ELS")

df = pd.DataFrame(metrics)
df.to_json('els_ndcg.json')
x = dp.sum_relevance(metrics['els_ndcg_k']) / len(metrics['els_ndcg_k'])
print(x)
df

0.22476630444674645


,query_id,true_dcg_k,els_dcg_k,els_ndcg_k
0,1,24.751266,7.018690,0.283569
1,2,20.037305,4.471278,0.223148
2,33,6.462113,1.204120,0.186335
3,56,14.764365,1.333333,0.090308
4,57,18.633786,3.333333,0.178887
5,109,5.227311,1.500000,0.286954
6,130,7.489170,1.500000,0.200289
7,171,3.200253,1.204120,0.376258
8,196,12.601480,3.819367,0.303089
9,214,7.297966,0.867194,0.118827


In [16]:
metrics = ev.recall_precision(dataset, documents, "ELS")
df = pd.DataFrame(metrics)
df.to_json('els_recall_precision.json')
df
dp.sum_relevance(metrics['f_scores']) / len(metrics['f_scores'])

0.195247491638796